In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd

from bs4 import BeautifulSoup
import requests

import tekore as tk
import asyncio

import time
import re



## Spotify API Authentication

In [5]:
client_id = '25035de4cc4b4862baffb133c232efa3'
client_secret = 'ddff264cbeb748019cd112e20494b17b'

### Spotipy

In [6]:
# auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(auth_manager=auth_manager)

## Tokore 
#### Chunked turned on to by pass the request limit of 100

In [7]:
token = tk.request_client_token(client_id, client_secret)
#sender = tk.RetryingSender(sender=tk.AsyncSender())
     #sender=sender,
spotify = tk.Spotify(token, chunked_on = True, max_limits_on=True)

### Connect playlist

In [31]:
link_playlist = 'https://open.spotify.com/playlist/0aXnOizaJ0eM1YIIcsSXSO'
playlist_id = '0aXnOizaJ0eM1YIIcsSXSO'

In [14]:
#Spotify created playlist
link_playlist = 'https://open.spotify.com/playlist/37i9dQZF1DX56bqlsMxJYR'

In [32]:

playlist = spotify.playlist(playlist_id)

In [6]:
first_items = spotify.playlist_items(playlist_id).items
tracks = []
#first_tracks = spotify.playlist_tracks(playlist_uri)

#### Exploring Tekore notation

In [8]:
playlist.name

'Best Hip-Hop Playlist'

In [33]:
playlist.id

'0aXnOizaJ0eM1YIIcsSXSO'

In [9]:
playlist.tracks.items[0].track.name

'Codeine Dreaming (feat. Lil Wayne)'

In [10]:
playlist.tracks.items[0].track

FullPlaylistTrack with fields:
  album = SimpleAlbum(album_group, album_type, artists, ...)
  artists = [2 x SimpleArtist(external_urls, href, id, name, type, uri)]
  available_markets = [183 x str]
  disc_number = 1
  duration_ms = 264531
  episode = False
  explicit = True
  external_ids = {'isrc'}
  external_urls = {'spotify'}
  href = 'https://api.spotify.com/v1/tracks/4DTpngLjoHj5gFxEZFeD3J'
  id = '4DTpngLjoHj5gFxEZFeD3J'
  is_local = False
  is_playable = None
  linked_from = None
  name = 'Codeine Dreaming (feat. Lil Wayne)'
  popularity = 66
  preview_url = 'https://p.scdn.co/mp3-preview/f0bcb280d4f87b0851e3e815...'
  restrictions = None
  track = True
  track_number = 20
  type = 'track'
  uri = 'spotify:track:4DTpngLjoHj5gFxEZFeD3J'

In [18]:
playlist.tracks.items[0].track.album.name

'Project Baby 2: All Grown Up (Deluxe Edition)'

In [142]:
playlist.tracks.items[0].track.artists[0].id

'46SHBwWsqBkxI7EeeBEQG7'

In [218]:
playlist.name

'Best Hip-Hop Playlist'

##### Testing album images

In [140]:
playlist.tracks.items[0].track.album.images[0].url

'https://i.scdn.co/image/ab67616d0000b2738e3046e13f253a208519b21a'

In [61]:
playlist.tracks.items[0].track.album.id

'0D2kFxAO1YPAuxAtP23g0p'

##### Featured artist testing

In [75]:
len(playlist.tracks.items[19].track.artists)

4

In [33]:
ft_art = []
for i in range(1,len(playlist.tracks.items[64].track.artists)):
    ft_art.append(playlist.tracks.items[64].track.artists[i].name)

ft_art

x = ', '.join(ft_art)
print(x)

In [72]:
playlist.tracks.items[19].track.artists[3]

SimpleArtist with fields:
  external_urls = {'spotify'}
  href = 'https://api.spotify.com/v1/artists/53KwLdlmrlCelAZMaLVZqU'
  id = '53KwLdlmrlCelAZMaLVZqU'
  name = 'James Blake'
  type = 'artist'
  uri = 'spotify:artist:53KwLdlmrlCelAZMaLVZqU'

In [102]:
len(playlist.tracks.items[0].track.artists)

2

In [83]:
playlist.tracks.items[1].track.artists[0].name

'21 Savage'

In [22]:
playlist.id

'0aXnOizaJ0eM1YIIcsSXSO'

#### Audio features

In [41]:
playlist.tracks.items[1].track.id

'2fQrGHiQOvpL9UgPvtYy6G'

In [44]:
af = spotify.track_audio_features('2fQrGHiQOvpL9UgPvtYy6G')

In [47]:
af.acousticness

0.015

In [152]:
# No followers
playlist.tracks.items[0].track.artists[0]

SimpleArtist with fields:
  external_urls = {'spotify'}
  href = 'https://api.spotify.com/v1/artists/46SHBwWsqBkxI7EeeBEQG7'
  id = '46SHBwWsqBkxI7EeeBEQG7'
  name = 'Kodak Black'
  type = 'artist'
  uri = 'spotify:artist:46SHBwWsqBkxI7EeeBEQG7'

In [157]:
playlist.tracks.items[0].track

FullPlaylistTrack with fields:
  album = SimpleAlbum(album_group, album_type, artists, ...)
  artists = [2 x SimpleArtist(external_urls, href, id, name, type, uri)]
  available_markets = [183 x str]
  disc_number = 1
  duration_ms = 264531
  episode = False
  explicit = True
  external_ids = {'isrc'}
  external_urls = {'spotify'}
  href = 'https://api.spotify.com/v1/tracks/4DTpngLjoHj5gFxEZFeD3J'
  id = '4DTpngLjoHj5gFxEZFeD3J'
  is_local = False
  is_playable = None
  linked_from = None
  name = 'Codeine Dreaming (feat. Lil Wayne)'
  popularity = 65
  preview_url = 'https://p.scdn.co/mp3-preview/f0bcb280d4f87b0851e3e815...'
  restrictions = None
  track = True
  track_number = 20
  type = 'track'
  uri = 'spotify:track:4DTpngLjoHj5gFxEZFeD3J'

## First Scrape test

In [33]:
# track name, track uri, track popularity, album name, album image, artist name, artist uri,
# artist follwers, artist popularity, artist genres
# danceability, engery, key, loudness, mode, speechiness, acousticness, instrumentalness,
# liveness, valence, tempo, duration_ms,

In [9]:
track_name = []
track_id = []
track_pop = []
artist_name = []
featured_artist = []
album_name = []
#album_cover = []
artist_id = []
#artist_followers = []
#artist_genres = []

danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
track_lyrics = []
playlist_name = []

for item in spotify.all_items(spotify.playlist_items(playlist_id)):
#looping through each track in the playlist

    track_name.append(item.track.name)
    track_id.append(item.track.id)
    track_pop.append(item.track.popularity)
    artist_name.append(item.track.artists[0].name)
    
    
    if (len(item.track.artists)) > 1:
        ft_art = []
        for i in range(1,len(item.track.artists)):
            ft_art.append(item.track.artists[i].name)
        x = ', '.join(ft_art)
    else:
         x ='None'
    featured_artist.append(x)
    
    album_name.append(item.track.album.name)
    #album_cover.append(item.track.album.images[0].url)
    artist_id.append(item.track.artists[0].id)
    
    af = spotify.track_audio_features(item.track.id)
    danceability.append(af.danceability)
    energy.append(af.energy)
    key.append(af.key)
    loudness.append(af.loudness)
    mode.append(af.mode)
    speechiness.append(af.speechiness)
    acousticness.append(af.acousticness)
    instrumentalness.append(af.instrumentalness)
    liveness.append(af.liveness)
    valence.append(af.valence)
    tempo.append(af.tempo)
                        #Scrape_lyrics2 function below
    track_lyrics.append(scrape_lyrics2(item.track.name, item.track.artists[0].name, x))
    playlist_name.append(playlist.name)
    # Used sleep to not exceed the number of requests in a given time
    time.sleep(.07)

    

df = pd.DataFrame(
    {'track_name': track_name,
    'track_id': track_id,
     'track_pop': track_pop,
     'artist_name': artist_name,
     'featured_artist': featured_artist,
     'album_name': album_name,
     #'album_cover': album_cover,
     'artist_id': artist_id,
     'danceability': danceability,
     'energy': energy,
     'key': key,
     'loudness': loudness,
     'mode': mode,
     'speechiness': speechiness,
     'acousticness': acousticness,
     'intrumentalness': instrumentalness,
     'valence': valence,
     'tempo': tempo,
     'track_lyrics': track_lyrics,
     'playlist_name': playlist_name
    })
    
    
  

In [10]:
df.shape

(1941, 19)

In [219]:
df['playlist_name'] = playlist.name

##### Saving the first playlist data

In [222]:
checkpoint = df.to_csv('datasets/checkpoint.csv')

### Testing for the "scrape_lyrics2" function used above

In [148]:
df['track_lyrics'][1]

'\t\t\tPlease check the spelling and try again to <a href="/index.php?section=search&amp;searchW=21 savage feat Bank account">search 21 savage feat Bank account lyrics</a>. <script type="text/javascript">var GOOG_FIXURL_LANG = "en";var GOOG_FIXURL_SITE = "https://www.songlyrics.com/";</script>. <script src="https://linkhelp.clients.google.com/tbproxy/lh/wm/fixurl.js" type="text/javascript"></script>. </p>'

In [75]:
playlist.tracks.items[0].track.artists[0].name

'Kodak Black'

In [85]:
songname = playlist.tracks.items[0].track.name
songname = songname.split(' (')
songname = songname[0]
songname = songname.replace(' ','-') if ' ' in songname else str(songname)
type(songname)
#songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
#songname2

str

In [87]:
artistname = playlist.tracks.items[0].track.artists[0].name
artistname = artistname.replace(' ','-') if ' ' in artistname else artistname
type(artistname)

str

### Lyric Scrape Testing
Reference : "https://medium.com/swlh/how-to-leverage-spotify-api-genius-lyrics-for-data-science-tasks-in-python-c36cdfb55cf3https://medium.com/swlh/how-to-leverage-spotify-api-genius-lyrics-for-data-science-tasks-in-python-c36cdfb55cf3"

In [11]:
artist = playlist.tracks.items[0].track.artists[0].name
song = playlist.tracks.items[0].track.name
if '-' in song:
        song = song.split('-')[0]
song = song.replace(' ','').replace("'",'').lower()
    #artist =track['track']['artists'][0]['name']
artist = artist.replace(' ', '').lower()
page = requests.get('https://www.azlyrics.com/'+ 'lyrics/' + artist + '/' + song + '.' + 'html')
html = BeautifulSoup(page.text, 'html.parser')
#print(html)
lyrics = html.find("div", class_="$0")
print(lyrics)

None


In [41]:
# https://www.songlyrics.com/kodak-black-feat-lil-wayne/codeine-dreaming-lyrics/
# https://www.songlyrics.com/metro-boomin-feat-offset-drake/no-complaints-lyrics/

In [21]:
playlist.tracks.items[64].track.artists

ModelList with items: [
  SimpleArtist(external_urls, href, id, name, type, uri)
  SimpleArtist(external_urls, href, id, name, type, uri)
  SimpleArtist(external_urls, href, id, name, type, uri)
]

In [181]:
ft_art = []
for i in range(1,len(playlist.tracks.items[1].track.artists)):
    ft_art.append(playlist.tracks.items[1].track.artists[i].name)

In [182]:
 x = ', '.join(ft_art)

In [183]:
x

''

##### Genius and AZ lyrics prohibited 3rd party use of the data. Used songlyrics.com instead.

In [180]:
#formatting url to fit artist and song
x = x
lyric_url = 'https://www.songlyrics.com/'
artist = (playlist.tracks.items[0].track.artists[0].name).replace(' ','-').lower()
song = (playlist.tracks.items[0].track.name).replace(' ','-').lower() + '-lyrics/'

ft_art =[f.replace(' ','-') for f in ft_art]
ft_art

feats = []
for f in ft_art:
    f = '-' + f
    feats.append(f)

string3 = ''
artist2 = string3.join(feats).lower()

if x == 'None':
    page_url = lyric_url + artist + '/' + song
else:
    page_url = lyric_url + artist + '-feat' + artist2 + '/' + song


#page_url = lyric_url + artist + '/' + song
print(page_url)

https://www.songlyrics.com/kodak-black-feat-lil-wayne/codeine-dreaming-(feat.-lil-wayne)-lyrics/


In [123]:
page = requests.get(page_url)
html = BeautifulSoup(page.text, 'html.parser')

#print(html)
#lyrics = html.find("div", class_="$0")
lyrics = html.find("p", class_="songLyricsV14 iComment-text")
lyrics = str(lyrics).split('\n')
#lyrics = lyrics.split('\n')
lyrics = lyrics[1:]
lyrics = [item.replace('<br/>','') for item in lyrics]
string1 = '. '
lyrics = string1.join(lyrics)

#p class="songLyricsV14 iComment-text" id="songLyricsDiv"
print(lyrics)

If a nigga hatin', call him Joe Budden (pussy). Coupe outside and it's press button (skrt skrt). Got that nigga mad 'cause your bitch fuckin' (hah). Fuck it, ice tray the gang (fuck it), ice tray the gang (woo). Ice tray the gang (ice), ice tray the gang. Fuck it, ice tray the gang (woo), ice tray the gang (ice). Ice tray the gang (yeah), ice tray the gang, fuck it. . Alexander Wang, wrist Eddie Cane. Drop top McLaren, park it at the Flame. Think I was insane, what my jeweler bring. Higher than Kurt Cobain, Huncho Bruce Wayne. Big boy Patek and it's skelly. Six ringtones on the celly. You did your ice wrong, go to Elliot. Splash on the wrist, J Reddick. I got a bag of drugs no medic. These niggas drinking on straight synthetic. I ain't tryna fuck no bitch that's petty. Skrt on the ho like Andrettis. It'll lock up if you wet it. Choppin' it up like machete. Swerve in the Junction, Spaghetti (swerve). Come to the Nawf if you ready (Nawf). If 'Set say he set then I set it ('Set). If Take'

##### Different variations of scrape function

In [167]:
def scrape_lyrics(song, artist, ft_art):
    #format url to follow path with artists followed by song name
    lyric_url = 'https://www.songlyrics.com/'
    artist = artist.replace(' ','-').lower()
    song = song.replace(' ','-').lower() + '-lyrics/'
    
    # Remove the (feat. artist) in song titles
    if '(' in song:
        song = re.sub(r'(?<=\().*?(?=\))', '', song).replace('-()','')
    #ft_art already has featured artitst 
    ft_art =[f.replace(' ','-') for f in ft_art]
    ft_art

    feats = []
    for f in ft_art:
        f = '-' + f
        feats.append(f)

    string3 = ''
    artist2 = string3.join(feats).lower()

    if x == 'None':
        page_url = lyric_url + artist + '/' + song
    else:
        page_url = lyric_url + artist + '-feat' + artist2 + '/' + song
    #artists = artist + artist2

    #lyric_url = 'https://www.songlyrics.com/'

    #page_url = lyric_url + artist + '/' + song
    
    #Use beautful soup tp scrape lyrics
    page = requests.get(page_url)
    html = BeautifulSoup(page.text, 'html.parser')

   
    lyrics = html.find("p", class_="songLyricsV14 iComment-text")
    lyrics = str(lyrics).split('\n')
    lyrics = lyrics[1:]
    lyrics = [item.replace('<br/>','') for item in lyrics]
    string1 = '. '
    lyrics = string1.join(lyrics)
    return lyrics

In [186]:
song = playlist.tracks.items[0].track.name
artist = playlist.tracks.items[0].track.artists[0].name
print(song)
print(artist)

Codeine Dreaming (feat. Lil Wayne)
Kodak Black


# Scrape Lyrics from songlyrics.com

In [72]:
def scrape_lyrics2(song, artist, x):
    #format url to follow path with artists followed by song name
    lyric_url = 'https://www.songlyrics.com/'
    artist = artist.replace(' ','-').lower()
    song = song.replace(' ','-').lower() + '-lyrics/'
    
    # Remove the (feat. artist) in song titles
    if '(' in song:
        song = re.sub(r'(?<=\().*?(?=\))', '', song).replace('-()','').replace('.','')

    artist2 = x.replace(', ','-').replace(' ', '-')

    if x == 'None':
        page_url = lyric_url + artist + '/' + song
    else:
        page_url = lyric_url + artist + '-feat-' + artist2 + '/' + song
   
    
    #Use beautful soup tp scrape lyrics
    try:
        page = requests.get(page_url, allow_redirects=1000)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics = html.find("p", class_="songLyricsV14 iComment-text")
        lyrics = str(lyrics).split('\n')
        lyrics = lyrics[1:]
        lyrics = [item.replace('<br/>','') for item in lyrics]
        string1 = '. '
        lyrics = string1.join(lyrics)
    
    except requests.exceptions.TooManyRedirects as exc:
        lyrics = 'None'
        
    return lyrics

In [210]:
scrape_lyrics2(playlist.tracks.items[0].track.name, playlist.tracks.items[0].track.artists[0].name, x)

URL:  https://www.songlyrics.com/kodak-black-feat-Lil-Wayne/codeine-dreaming-lyrics/


"Take that all the way up. I'm about to take off on these niggas. And I'm codeine dreaming. (and I'm codeine dreaming,and I'm codeine dreaming. And I'm codeine dreaming,and I'm codeine dreaming). . I might take off on you peons and go away to Venus. I'm a star so I put a neutron on my pinkie. I'm moonwalking but my Christian Loubotin's bleeding. I'm smoking moonrocks, shawty, and I'm feeling seated. I might take off on you peons and go away to Venus. I'm a star so I put a neutron on my pinkie. I'm moonwalking but my Christian Loubotin's bleeding. I'm smoking moonrocks, shawty, and I'm feeling seated. . . I'm in Uranus right now, I think I've seen a demon. I thought I left you back there in the cell or am I dreamin'?. Say I ain't dealing with your dawg, going a different way. I broke up with my redbone, I went the milky way. I'm ridin' in a Wraith feeling like I'm in outer space. Stars on my sunroof, Planet of the Apes. I couldn't wait until they released me, right back on the turf. I t

In [159]:
artist = playlist.tracks.items[0].track.artists[0].name.replace(' ','-').lower()
song = playlist.tracks.items[0].track.name.replace(' ','-').lower() + '-lyrics/'
print(artist)
print(song)

kodak-black
codeine-dreaming-(feat.-lil-wayne)-lyrics/


In [162]:
song = re.sub(r'(?<=\().*?(?=\))', '', song).replace('-()','')
print(song)

codeine-dreaming-lyrics/


#### More testing for the above function without running the whole script

In [166]:
    x = 'None'
    lyric_url = 'https://www.songlyrics.com/'
    artist = playlist.tracks.items[1].track.artists[0].name.replace(' ','-').lower()
    song = playlist.tracks.items[1].track.name.replace(' ','-').lower() + '-lyrics/'
    
    if '(' in song:
        song = re.sub(r'(?<=\().*?(?=\))', '', song).replace('-()','')
    #ft_art already has featured artitst 
    ft_art =[f.replace(' ','-') for f in ft_art]
    ft_art

    feats = []
    for f in ft_art:
        f = '-' + f
        feats.append(f)

    string3 = ''
    artist2 = string3.join(feats).lower()

    if x == 'None':
        page_url = lyric_url + artist + '/' + song
    else:
        page_url = lyric_url + artist + '-feat' + artist2 + '/' + song
        
    
    #artists = artist + artist2

    #lyric_url = 'https://www.songlyrics.com/'

    #page_url = lyric_url + artist + '/' + song
    
    #Use beautful soup tp scrape lyrics
    page = requests.get(page_url)
    html = BeautifulSoup(page.text, 'html.parser')
    
   
    lyrics = html.find("p", class_="songLyricsV14 iComment-text")
    lyrics = str(lyrics).split('\n')
    lyrics = lyrics[1:]
    lyrics = [item.replace('<br/>','') for item in lyrics]
    string1 = '. '
    lyrics = string1.join(lyrics)
    print(lyrics)

I tear down the mall with the bitch (for real). You can't even talk to the bitch (no). She fucking with bosses and shit (oh God). I pull up in 'rari's and shit, with choppers and Harley's and shit (for real). I be Gucci'd down, you wearing Lacoste and shit (bitch). Yeah, Moncler, yeah, fur came off of that, yeah (yeah). Triple homicide, put me in a chair, yeah (in jail). Triple cross the plug, we do not play fair, yeah (Oh god). Got 'em tennis chains on and they real blingy (blingy). Draco make you do the chicken head like Chingy (Chingy). Walk in Neiman Marcus and I spend a light fifty (fifty). Please proceed with caution, shooters, they be right with me (21). Bad bitch, cute face and some nice titties. $7500 on a Saint Laurent jacket (yeah). Bitch, be careful where you dumpin' your ashes (bitch). I ain't no sucker, I ain't cut for no action (nah). The skreets raised me, I'm a ho bastard (wild, wild, wild, wild). I bought a 'Rari just so I can go faster (skrrt). Niggas tryna copy me, 

(1941, 19)

In [226]:
# These are playlist Ids from random playlists on spotofy 
test = [
    '1KNl4AYfgZtOVm9KHkhPTF',
    '2H11P9KofhWu9EYUOoqOPm',
    '0pM9c17S6YMUiFw6Q2jLvp'
]

# Scrape Multiple Playlists

In [3]:
def scrape_mult(soundtracks):
    global df
    for st in soundtracks:
        
        playlist = spotify.playlist(st)
        
        track_name = []
        track_id = []
        track_pop = []
        artist_name = []
        featured_artist = []
        album_name = []
        artist_id = []
        danceability = []
        energy = []
        key = []
        loudness = []
        mode = []
        speechiness = []
        acousticness = []
        instrumentalness = []
        liveness = []
        valence = []
        tempo = []
        track_lyrics = []
        playlist_name = []
        playlist_id = []
        
        for item in spotify.all_items(spotify.playlist_items(st)):
            track_name.append(item.track.name)
            track_id.append(item.track.id)
            track_pop.append(item.track.popularity)
            artist_name.append(item.track.artists[0].name)
    
    
            if (len(item.track.artists)) > 1:
                ft_art = []
                for i in range(1,len(item.track.artists)):
                    ft_art.append(item.track.artists[i].name)
                x = ', '.join(ft_art)
            else:
                 x ='None'
            featured_artist.append(x)
    
            album_name.append(item.track.album.name)
        #album_cover.append(item.track.album.images[0].url)
            artist_id.append(item.track.artists[0].id)
        #last check
    
            af = spotify.track_audio_features(item.track.id)
            danceability.append(af.danceability)
            energy.append(af.energy)
            key.append(af.key)
            loudness.append(af.loudness)
            mode.append(af.mode)
            speechiness.append(af.speechiness)
            acousticness.append(af.acousticness)
            instrumentalness.append(af.instrumentalness)
            liveness.append(af.liveness)
            valence.append(af.valence)
            tempo.append(af.tempo)
            track_lyrics.append(scrape_lyrics2(item.track.name, item.track.artists[0].name, x))
            playlist_name.append(playlist.name)
            playlist_id.append(playlist.id)
            time.sleep(.07)
        
    

            df2 = pd.DataFrame(
            {'track_name': track_name,
            'track_id': track_id,
             'track_pop': track_pop,
             'artist_name': artist_name,
             'featured_artist': featured_artist,
             'album_name': album_name,
             #'album_cover': album_cover,
             'artist_id': artist_id,
             'danceability': danceability,
             'energy': energy,
             'key': key,
             'loudness': loudness,
             'mode': mode,
             'speechiness': speechiness,
             'acousticness': acousticness,
             'intrumentalness': instrumentalness,
             'valence': valence,
             'tempo': tempo,
             'track_lyrics': track_lyrics,
             'playlist_name': playlist_name,
             'playlist_id': playlist_id
            })
        
        df = df.append(df2, ignore_index=True)
    #df.to_csv('datasets/checkpoint' + "_" + len(df) +'.csv')
    return df.head()

In [ ]:
# using dataframe and loop through song and artist and scrape lyrics seperately, and have seperate sleep. Merge together

In [236]:
scrape_mult(test)

C:\Users\brice\AppData\Local\Temp\ipykernel_30668\2814680714.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)


,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,key,loudness,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name
0,Codeine Dreaming (feat. Lil Wayne),4DTpngLjoHj5gFxEZFeD3J,66,Kodak Black,Lil Wayne,Project Baby 2: All Grown Up (Deluxe Edition),46SHBwWsqBkxI7EeeBEQG7,0.736,0.604,1,-7.357,0,0.222,0.0652,0.000064,0.574,155.007,Take that all the way up. I'm about to take of...,Best Hip-Hop Playlist
1,Bank Account,2fQrGHiQOvpL9UgPvtYy6G,18,21 Savage,None,Issa Album,1URnnhqYAYcrqrcwql10ft,0.884,0.347,8,-8.227,0,0.350,0.0150,0.000007,0.376,75.016,I tear down the mall with the bitch (for real)...,Best Hip-Hop Playlist
2,PICK IT UP (feat. A$AP Rocky),3ncgNpxLoBQ65ABk4djDyd,66,Famous Dex,A$AP Rocky,Pick It Up (feat. A$AP Rocky),0WOxhx4hikIsyF3CRPLC8W,0.921,0.467,1,-8.443,1,0.119,0.0149,0.000238,0.287,135.995,"That's my fucking brother right there, ya dig?...",Best Hip-Hop Playlist
3,Ric Flair Drip (with Metro Boomin),7sO5G9EABYOXQKNPNiE9NR,82,Offset,Metro Boomin,Without Warning,4DdkRBBYG6Yk9Ka8tdJ9BW,0.880,0.428,9,-8.280,1,0.206,0.1490,0.000051,0.333,100.007,"Metro Boomin' want some more, nigga!. . Going ...",Best Hip-Hop Playlist
4,Roll in Peace (feat. XXXTENTACION),40oKW22ZNNkEdZLJTScaQI,74,Kodak Black,XXXTENTACION,Project Baby 2: All Grown Up,46SHBwWsqBkxI7EeeBEQG7,0.747,0.524,10,-6.807,0,0.245,0.0306,0.000000,0.363,140.053,"(We got London on the track!). These days, I d...",Best Hip-Hop Playlist


In [237]:
df.shape

(2141, 19)

In [238]:
df.tail()

,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,key,loudness,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name
2136,I'm Gonna Be (500 Miles),67iAlVNDDdddxqSD2EZhFs,74,The Proclaimers,None,Sunshine on Leith,1A92IAcd7A6npCA33oGM5i,0.851,0.551,4,-5.177,1,0.0396,0.1510,0.000000,0.807,131.922,Miscellaneous. I'm Gonna Be. When I wake up we...,Top 50 Happy Songs
2137,We Are Family - 1995 Remaster,5IKLwqBQG6KU6MP2zP80Nu,72,Sister Sledge,None,We Are Family (1995 Remaster),6gkWznnJkdkwRPVcmnrays,0.784,0.893,7,-5.153,0,0.0923,0.3000,0.000016,0.819,118.738,\t\t\tPlease check the spelling and try again ...,Top 50 Happy Songs
2138,"I'll Be There for You - Theme From ""Friends""",15tHagkk8z306XkyOHqiip,69,The Rembrandts,None,L.P.,0gDg7FEsF4Y1jWddJJgcn4,0.320,0.917,9,-5.903,1,0.0771,0.0717,0.000000,0.715,190.757,"Your job's a joke, you're broke, your love lif...",Top 50 Happy Songs
2139,Ain't Nobody,2NVpYQqdraEcQwqT7GhUkh,68,Chaka Khan,None,"Epiphany: The Best of Chaka Khan, Vol. 1",6mQfAAqZGBzIfrmlZCeaYT,0.800,0.878,1,-7.109,1,0.0434,0.1890,0.000104,0.816,104.282,That's the way it was. Happened so naturally. ...,Top 50 Happy Songs
2140,You Sexy Thing,714hERk9U1W8FMYkoC83CO,73,Hot Chocolate,None,Hot Chocolate,72VzFto8DYvKHocaHYNWSi,0.791,0.733,5,-5.465,1,0.0633,0.5230,0.001600,0.962,105.853,Miscellaneous. You Sexy Thing. I believe in mi...,Top 50 Happy Songs


In [244]:

#df.to_csv('datasets/checkpoint' + "-" + datetime.now().strftime('%Y-%m-%d-%H-%M-%S')+'.csv')

In [261]:
soundtracks = [
    '2b4Jpy0qpbGk2cIF0ikGTD',
    '3CDX7S8NcO0y3FuKIJEfSN',
    '0JiVp7Z0pYKI8diUV6HJyQ',
    '3XM4qNNOrn2PcaiyIe8nax',
    '2rKy7TRijAqjKXoG7wDeuj',
    '1BJTY5xJdhbPmS9Os2s9vf',
    '5Rooa8nylHIiBOzP7PugBq',
    '4nDffrxlJOBkmJlOmxkjXg',
    '1KNl4AYfgZtOVm9KHkhPTF',
    '2H11P9KofhWu9EYUOoqOPm',
    '0pM9c17S6YMUiFw6Q2jLvp',
]

In [264]:
#Only used maybe half of these playlist, trouble shooting errors and trying smaller amounts
scrape_mult(soundtracks)

C:\Users\brice\AppData\Local\Temp\ipykernel_30668\2790555846.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)


,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,key,loudness,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name
0,Codeine Dreaming (feat. Lil Wayne),4DTpngLjoHj5gFxEZFeD3J,66,Kodak Black,Lil Wayne,Project Baby 2: All Grown Up (Deluxe Edition),46SHBwWsqBkxI7EeeBEQG7,0.736,0.604,1,-7.357,0,0.222,0.0652,0.000064,0.574,155.007,Take that all the way up. I'm about to take of...,Best Hip-Hop Playlist
1,Bank Account,2fQrGHiQOvpL9UgPvtYy6G,18,21 Savage,None,Issa Album,1URnnhqYAYcrqrcwql10ft,0.884,0.347,8,-8.227,0,0.350,0.0150,0.000007,0.376,75.016,I tear down the mall with the bitch (for real)...,Best Hip-Hop Playlist
2,PICK IT UP (feat. A$AP Rocky),3ncgNpxLoBQ65ABk4djDyd,66,Famous Dex,A$AP Rocky,Pick It Up (feat. A$AP Rocky),0WOxhx4hikIsyF3CRPLC8W,0.921,0.467,1,-8.443,1,0.119,0.0149,0.000238,0.287,135.995,"That's my fucking brother right there, ya dig?...",Best Hip-Hop Playlist
3,Ric Flair Drip (with Metro Boomin),7sO5G9EABYOXQKNPNiE9NR,82,Offset,Metro Boomin,Without Warning,4DdkRBBYG6Yk9Ka8tdJ9BW,0.880,0.428,9,-8.280,1,0.206,0.1490,0.000051,0.333,100.007,"Metro Boomin' want some more, nigga!. . Going ...",Best Hip-Hop Playlist
4,Roll in Peace (feat. XXXTENTACION),40oKW22ZNNkEdZLJTScaQI,74,Kodak Black,XXXTENTACION,Project Baby 2: All Grown Up,46SHBwWsqBkxI7EeeBEQG7,0.747,0.524,10,-6.807,0,0.245,0.0306,0.000000,0.363,140.053,"(We got London on the track!). These days, I d...",Best Hip-Hop Playlist


In [265]:
df.shape

(7253, 19)

In [268]:
df = df.drop_duplicates()

In [271]:
df.to_csv('datasets/checkpoint2.csv')

In [275]:
soundtracks = [
    '4RVf1hHtwvMEED3yuCNi8q'
]

In [276]:
scrape_mult(soundtracks)

C:\Users\brice\AppData\Local\Temp\ipykernel_30668\131950147.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)


,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,key,loudness,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name
0,Codeine Dreaming (feat. Lil Wayne),4DTpngLjoHj5gFxEZFeD3J,66,Kodak Black,Lil Wayne,Project Baby 2: All Grown Up (Deluxe Edition),46SHBwWsqBkxI7EeeBEQG7,0.736,0.604,1,-7.357,0,0.222,0.0652,0.000064,0.574,155.007,Take that all the way up. I'm about to take of...,Best Hip-Hop Playlist
1,Bank Account,2fQrGHiQOvpL9UgPvtYy6G,18,21 Savage,None,Issa Album,1URnnhqYAYcrqrcwql10ft,0.884,0.347,8,-8.227,0,0.350,0.0150,0.000007,0.376,75.016,I tear down the mall with the bitch (for real)...,Best Hip-Hop Playlist
2,PICK IT UP (feat. A$AP Rocky),3ncgNpxLoBQ65ABk4djDyd,66,Famous Dex,A$AP Rocky,Pick It Up (feat. A$AP Rocky),0WOxhx4hikIsyF3CRPLC8W,0.921,0.467,1,-8.443,1,0.119,0.0149,0.000238,0.287,135.995,"That's my fucking brother right there, ya dig?...",Best Hip-Hop Playlist
3,Ric Flair Drip (with Metro Boomin),7sO5G9EABYOXQKNPNiE9NR,82,Offset,Metro Boomin,Without Warning,4DdkRBBYG6Yk9Ka8tdJ9BW,0.880,0.428,9,-8.280,1,0.206,0.1490,0.000051,0.333,100.007,"Metro Boomin' want some more, nigga!. . Going ...",Best Hip-Hop Playlist
4,Roll in Peace (feat. XXXTENTACION),40oKW22ZNNkEdZLJTScaQI,74,Kodak Black,XXXTENTACION,Project Baby 2: All Grown Up,46SHBwWsqBkxI7EeeBEQG7,0.747,0.524,10,-6.807,0,0.245,0.0306,0.000000,0.363,140.053,"(We got London on the track!). These days, I d...",Best Hip-Hop Playlist


In [277]:
df.shape

(10601, 19)

In [18]:
df = df.drop_duplicates()
df.shape

(5617, 20)

In [4]:
#organizing which playlist were attempted to scrape and which ones went through
soundtracks = [
    '3ahkwrXrfObks9j6g1XmBd',
    '53Bf0yOM2uafN1ZUqCmtPE',
    '6DsHDZmrOgLBmxa9DJE6sb',
    '5Rooa8nylHIiBOzP7PugBq',
    '4nDffrxlJOBkmJlOmxkjXg',
    '1KNl4AYfgZtOVm9KHkhPTF',
    '2H11P9KofhWu9EYUOoqOPm',
    '0pM9c17S6YMUiFw6Q2jLvp',
    '2b4Jpy0qpbGk2cIF0ikGTD',
    '3CDX7S8NcO0y3FuKIJEfSN',
    '0JiVp7Z0pYKI8diUV6HJyQ',
    '3XM4qNNOrn2PcaiyIe8nax',
    '2rKy7TRijAqjKXoG7wDeuj',
    '1BJTY5xJdhbPmS9Os2s9vf',
    '5Rooa8nylHIiBOzP7PugBq',
    '4nDffrxlJOBkmJlOmxkjXg',
    '1KNl4AYfgZtOVm9KHkhPTF',
    '2H11P9KofhWu9EYUOoqOPm',
    '0pM9c17S6YMUiFw6Q2jLvp',
    '4RVf1hHtwvMEED3yuCNi8q'
]

In [1]:
#scrape_mult(soundtracks)

In [10]:
df = pd.read_csv('datasets/checkpoint2.csv', index_col=False)

In [11]:
df.shape

(4037, 20)

In [17]:
#df.shape

(356528, 20)

In [14]:
#df = df.drop_duplicates()

In [13]:
df.head(2)

,Unnamed: 0,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,key,loudness,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name
0,0,Codeine Dreaming (feat. Lil Wayne),4DTpngLjoHj5gFxEZFeD3J,66,Kodak Black,Lil Wayne,Project Baby 2: All Grown Up (Deluxe Edition),46SHBwWsqBkxI7EeeBEQG7,0.736,0.604,1,-7.357,0,0.222,0.0652,0.000064,0.574,155.007,Take that all the way up. I'm about to take of...,Best Hip-Hop Playlist
1,1,Bank Account,2fQrGHiQOvpL9UgPvtYy6G,18,21 Savage,None,Issa Album,1URnnhqYAYcrqrcwql10ft,0.884,0.347,8,-8.227,0,0.350,0.0150,0.000007,0.376,75.016,I tear down the mall with the bitch (for real)...,Best Hip-Hop Playlist


In [27]:
df2 = pd.read_csv('datasets/checkpoint.csv', index_col=False)

In [28]:
df2['playlist_name'].unique()

array(['Best Hip-Hop Playlist'], dtype=object)

### Troubleshot many errors, lost playlists in the process. Checking which playlists went through.

In [17]:
#Playlists used
df['playlist_name'].unique()

array(['Best Hip-Hop Playlist', 'Global Top 50 | 2022 Hits',
       'The Best Farewell Songs Top 50 Funeral Songs ',
       'Top 50 Happy Songs', 'Top 500 Classic Rock Tracks',
       'TOP 500: Best Club Hits of All Time',
       'Spotify Top 500 Most Streamed Songs of All Time'], dtype=object)

In [134]:
#df[df['playlist_name']=='Global Top 50 | 2022 Hits'][:5]

In [19]:
#looking for which playlist went through
def soundtrack_names(soundtracks):
    for st in soundtracks:
        playlist = spotify.playlist(st)
        print(playlist.name + ': ' + playlist.id)

In [25]:
soundtracks = [
    '0aXnOizaJ0eM1YIIcsSXSO',
    '1KNl4AYfgZtOVm9KHkhPTF',
    '2H11P9KofhWu9EYUOoqOPm',
    '3ahkwrXrfObks9j6g1XmBd',
    '53Bf0yOM2uafN1ZUqCmtPE',
    '6DsHDZmrOgLBmxa9DJE6sb',
    '5Rooa8nylHIiBOzP7PugBq',
    '4nDffrxlJOBkmJlOmxkjXg',
    '1KNl4AYfgZtOVm9KHkhPTF',
    '2H11P9KofhWu9EYUOoqOPm',
    '0pM9c17S6YMUiFw6Q2jLvp',
    '2b4Jpy0qpbGk2cIF0ikGTD',
    '3CDX7S8NcO0y3FuKIJEfSN',
    '0JiVp7Z0pYKI8diUV6HJyQ',
    '3XM4qNNOrn2PcaiyIe8nax',
    '2rKy7TRijAqjKXoG7wDeuj',
    '1BJTY5xJdhbPmS9Os2s9vf',
    '4nDffrxlJOBkmJlOmxkjXg',
    '2H11P9KofhWu9EYUOoqOPm',
    '0pM9c17S6YMUiFw6Q2jLvp',
    '4RVf1hHtwvMEED3yuCNi8q'
]

In [26]:
soundtrack_names(soundtracks)

Best Hip-Hop Playlist: 0aXnOizaJ0eM1YIIcsSXSO
Global Top 50 | 2022 Hits: 1KNl4AYfgZtOVm9KHkhPTF
The Best Farewell Songs Top 50 Funeral Songs : 2H11P9KofhWu9EYUOoqOPm
shower playlist #5000: 3ahkwrXrfObks9j6g1XmBd
Top 5000 Alternative Rock / Punk Rock / Emo 🌎: 53Bf0yOM2uafN1ZUqCmtPE
old songs (2000-2017): 6DsHDZmrOgLBmxa9DJE6sb
Top 500 happy songs sorted by happiness according to AI  : 5Rooa8nylHIiBOzP7PugBq
This is the playlist I listen to when I'm in no particular mood and I'm probably in my living room: 4nDffrxlJOBkmJlOmxkjXg
Global Top 50 | 2022 Hits: 1KNl4AYfgZtOVm9KHkhPTF
The Best Farewell Songs Top 50 Funeral Songs : 2H11P9KofhWu9EYUOoqOPm
Top 50 Happy Songs: 0pM9c17S6YMUiFw6Q2jLvp
Top 500 Classic Rock Tracks: 2b4Jpy0qpbGk2cIF0ikGTD
TOP 500: Best Club Hits of All Time: 3CDX7S8NcO0y3FuKIJEfSN
Spotify Top 500 Most Streamed Songs of All Time: 0JiVp7Z0pYKI8diUV6HJyQ
Top 500 Songs of the 2010's: 3XM4qNNOrn2PcaiyIe8nax
1980s - Billboard's Top 500 Songs: 2rKy7TRijAqjKXoG7wDeuj
TOP 500 SO

In [ ]:
# Best Hip-Hop Playlist', 'Global Top 50 | 2022 Hits',
#        'The Best Farewell Songs Top 50 Funeral Songs ',
#        'Top 50 Happy Songs', 'Top 500 Classic Rock Tracks',
#        'TOP 500: Best Club Hits of All Time',
#        'Spotify Top 500 Most Streamed Songs of All Time']

(1941, 20)

### Scrape lyrics from the dataframe after the tracks and audio features were obtained with tekore instead of in one function.

In [118]:

def scrape_lyrics_df(df):
    lyric = []
    for i in df.index:
        song = df['track_name'][i]
        song = song.replace(' ','-').lower() + '-lyrics/'
    
        # Remove the (feat. artist) in song titles
        if '(' in song:
            song = re.sub(r'(?<=\().*?(?=\))', '', song).replace('-()','').replace('.','')
        #print(song)
        
        artist = df['artist_name'][i]
        artist = artist.replace(' ','-').lower()
        
        artist2 = df['featured_artist'][i]
        artist2 = artist2.replace(', ','-').replace(' ', '-')
        artist2 = artist2.replace(' ', '-')
        
        lyric_url = 'https://www.songlyrics.com/' 
        if artist2 == 'None':
            page_url = lyric_url + artist + '/' + song
        else:
            page_url = lyric_url + artist + '-feat-' + artist2 + '/' + song
   
    
        #Use beautful soup tp scrape lyrics
        try:
            page = requests.get(page_url, allow_redirects=1000)
            html = BeautifulSoup(page.text, 'html.parser')
            lyrics = html.find("p", class_="songLyricsV14 iComment-text")
            lyrics = str(lyrics).split('\n')
            lyrics = lyrics[1:]
            lyrics = [item.replace('<br/>','') for item in lyrics]
            string1 = '. '
            lyrics = string1.join(lyrics)
    
        except requests.exceptions.TooManyRedirects as exc:
            lyrics = 'None'
        lyric.append(lyrics)
    df['lyrics'] = lyric
    return df.head(5)

In [6]:
#Vader

In [111]:
#scrape_lyrics_df(df)

In [107]:
df.shape

(4321, 21)

In [105]:
df[df['track_name'].isnull()]

,Unnamed: 0,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,...,loudness,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name,playlist_id
519,519.0,NaN,4FCXpbdc0SNfk742Eu0ub4,0,NaN,None,NaN,0LyfQWJT6nXafLPZqxe9Of,0.342,0.258,...,-11.885,0,0.0574,0.04500,0.007080,0.0459,97.327,NaN,Best Hip-Hop Playlist,NaN
522,522.0,NaN,3mBmNzIV5fpREYkjgr1OAI,0,NaN,None,NaN,0LyfQWJT6nXafLPZqxe9Of,0.718,0.504,...,-9.213,1,0.0382,0.00159,0.000005,0.0978,118.902,NaN,Best Hip-Hop Playlist,NaN
523,523.0,NaN,5VuCkJ8FsbxGMqlofL3ltH,0,NaN,None,NaN,0LyfQWJT6nXafLPZqxe9Of,0.613,0.465,...,-11.752,1,0.3270,0.17700,0.000000,0.5100,93.120,NaN,Best Hip-Hop Playlist,NaN
652,652.0,NaN,4Bw1hY1V7p6RffgeleQocf,0,NaN,None,NaN,0LyfQWJT6nXafLPZqxe9Of,0.511,0.454,...,-9.041,0,0.5240,0.20100,0.000093,0.2150,96.660,NaN,Best Hip-Hop Playlist,NaN
1086,1086.0,NaN,7EQoRKZQ1Y792aF7B6nPS6,0,NaN,None,NaN,0LyfQWJT6nXafLPZqxe9Of,0.849,0.397,...,-12.085,1,0.1340,0.40400,0.000002,0.0836,99.974,NaN,Best Hip-Hop Playlist,NaN


In [124]:
# df = df.dropna(subset=['track_name'], how='all')
# df.reset_index(inplace=True)


In [126]:
df.head(2)
df.drop(columns=['level_0', 'Unnamed: 0'], inplace=True)

In [127]:
df.head(2)

,index,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,...,loudness,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name,playlist_id
0,0,Codeine Dreaming (feat. Lil Wayne),4DTpngLjoHj5gFxEZFeD3J,66,Kodak Black,Lil Wayne,Project Baby 2: All Grown Up (Deluxe Edition),46SHBwWsqBkxI7EeeBEQG7,0.736,0.604,...,-7.357,0,0.222,0.0652,0.000064,0.574,155.007,Take that all the way up. I'm about to take of...,Best Hip-Hop Playlist,NaN
1,1,Bank Account,2fQrGHiQOvpL9UgPvtYy6G,18,21 Savage,None,Issa Album,1URnnhqYAYcrqrcwql10ft,0.884,0.347,...,-8.227,0,0.350,0.0150,0.000007,0.376,75.016,I tear down the mall with the bitch (for real)...,Best Hip-Hop Playlist,NaN


In [109]:
df.shape

(4316, 21)

In [113]:
df.head()

,Unnamed: 0,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,...,loudness,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name,playlist_id
0,0.0,Codeine Dreaming (feat. Lil Wayne),4DTpngLjoHj5gFxEZFeD3J,66,Kodak Black,Lil Wayne,Project Baby 2: All Grown Up (Deluxe Edition),46SHBwWsqBkxI7EeeBEQG7,0.736,0.604,...,-7.357,0,0.222,0.0652,0.000064,0.574,155.007,Take that all the way up. I'm about to take of...,Best Hip-Hop Playlist,NaN
1,1.0,Bank Account,2fQrGHiQOvpL9UgPvtYy6G,18,21 Savage,None,Issa Album,1URnnhqYAYcrqrcwql10ft,0.884,0.347,...,-8.227,0,0.350,0.0150,0.000007,0.376,75.016,I tear down the mall with the bitch (for real)...,Best Hip-Hop Playlist,NaN
2,2.0,PICK IT UP (feat. A$AP Rocky),3ncgNpxLoBQ65ABk4djDyd,66,Famous Dex,A$AP Rocky,Pick It Up (feat. A$AP Rocky),0WOxhx4hikIsyF3CRPLC8W,0.921,0.467,...,-8.443,1,0.119,0.0149,0.000238,0.287,135.995,"That's my fucking brother right there, ya dig?...",Best Hip-Hop Playlist,NaN
3,3.0,Ric Flair Drip (with Metro Boomin),7sO5G9EABYOXQKNPNiE9NR,82,Offset,Metro Boomin,Without Warning,4DdkRBBYG6Yk9Ka8tdJ9BW,0.880,0.428,...,-8.280,1,0.206,0.1490,0.000051,0.333,100.007,"Metro Boomin' want some more, nigga!. . Going ...",Best Hip-Hop Playlist,NaN
4,4.0,Roll in Peace (feat. XXXTENTACION),40oKW22ZNNkEdZLJTScaQI,74,Kodak Black,XXXTENTACION,Project Baby 2: All Grown Up,46SHBwWsqBkxI7EeeBEQG7,0.747,0.524,...,-6.807,0,0.245,0.0306,0.000000,0.363,140.053,"(We got London on the track!). These days, I d...",Best Hip-Hop Playlist,NaN


In [2]:
#df.iloc[518]

In [128]:
scrape_lyrics_df(df)


,index,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,...,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name,playlist_id,lyrics
0,0,Codeine Dreaming (feat. Lil Wayne),4DTpngLjoHj5gFxEZFeD3J,66,Kodak Black,Lil Wayne,Project Baby 2: All Grown Up (Deluxe Edition),46SHBwWsqBkxI7EeeBEQG7,0.736,0.604,...,0,0.222,0.0652,0.000064,0.574,155.007,Take that all the way up. I'm about to take of...,Best Hip-Hop Playlist,NaN,Take that all the way up. I'm about to take of...
1,1,Bank Account,2fQrGHiQOvpL9UgPvtYy6G,18,21 Savage,None,Issa Album,1URnnhqYAYcrqrcwql10ft,0.884,0.347,...,0,0.350,0.0150,0.000007,0.376,75.016,I tear down the mall with the bitch (for real)...,Best Hip-Hop Playlist,NaN,I tear down the mall with the bitch (for real)...
2,2,PICK IT UP (feat. A$AP Rocky),3ncgNpxLoBQ65ABk4djDyd,66,Famous Dex,A$AP Rocky,Pick It Up (feat. A$AP Rocky),0WOxhx4hikIsyF3CRPLC8W,0.921,0.467,...,1,0.119,0.0149,0.000238,0.287,135.995,"That's my fucking brother right there, ya dig?...",Best Hip-Hop Playlist,NaN,"That's my fucking brother right there, ya dig?..."
3,3,Ric Flair Drip (with Metro Boomin),7sO5G9EABYOXQKNPNiE9NR,82,Offset,Metro Boomin,Without Warning,4DdkRBBYG6Yk9Ka8tdJ9BW,0.880,0.428,...,1,0.206,0.1490,0.000051,0.333,100.007,"Metro Boomin' want some more, nigga!. . Going ...",Best Hip-Hop Playlist,NaN,"Metro Boomin' want some more, nigga!. . Going ..."
4,4,Roll in Peace (feat. XXXTENTACION),40oKW22ZNNkEdZLJTScaQI,74,Kodak Black,XXXTENTACION,Project Baby 2: All Grown Up,46SHBwWsqBkxI7EeeBEQG7,0.747,0.524,...,0,0.245,0.0306,0.000000,0.363,140.053,"(We got London on the track!). These days, I d...",Best Hip-Hop Playlist,NaN,"(We got London on the track!). These days, I d..."


In [129]:
df.head()

,index,track_name,track_id,track_pop,artist_name,featured_artist,album_name,artist_id,danceability,energy,...,mode,speechiness,acousticness,intrumentalness,valence,tempo,track_lyrics,playlist_name,playlist_id,lyrics
0,0,Codeine Dreaming (feat. Lil Wayne),4DTpngLjoHj5gFxEZFeD3J,66,Kodak Black,Lil Wayne,Project Baby 2: All Grown Up (Deluxe Edition),46SHBwWsqBkxI7EeeBEQG7,0.736,0.604,...,0,0.222,0.0652,0.000064,0.574,155.007,Take that all the way up. I'm about to take of...,Best Hip-Hop Playlist,NaN,Take that all the way up. I'm about to take of...
1,1,Bank Account,2fQrGHiQOvpL9UgPvtYy6G,18,21 Savage,None,Issa Album,1URnnhqYAYcrqrcwql10ft,0.884,0.347,...,0,0.350,0.0150,0.000007,0.376,75.016,I tear down the mall with the bitch (for real)...,Best Hip-Hop Playlist,NaN,I tear down the mall with the bitch (for real)...
2,2,PICK IT UP (feat. A$AP Rocky),3ncgNpxLoBQ65ABk4djDyd,66,Famous Dex,A$AP Rocky,Pick It Up (feat. A$AP Rocky),0WOxhx4hikIsyF3CRPLC8W,0.921,0.467,...,1,0.119,0.0149,0.000238,0.287,135.995,"That's my fucking brother right there, ya dig?...",Best Hip-Hop Playlist,NaN,"That's my fucking brother right there, ya dig?..."
3,3,Ric Flair Drip (with Metro Boomin),7sO5G9EABYOXQKNPNiE9NR,82,Offset,Metro Boomin,Without Warning,4DdkRBBYG6Yk9Ka8tdJ9BW,0.880,0.428,...,1,0.206,0.1490,0.000051,0.333,100.007,"Metro Boomin' want some more, nigga!. . Going ...",Best Hip-Hop Playlist,NaN,"Metro Boomin' want some more, nigga!. . Going ..."
4,4,Roll in Peace (feat. XXXTENTACION),40oKW22ZNNkEdZLJTScaQI,74,Kodak Black,XXXTENTACION,Project Baby 2: All Grown Up,46SHBwWsqBkxI7EeeBEQG7,0.747,0.524,...,0,0.245,0.0306,0.000000,0.363,140.053,"(We got London on the track!). These days, I d...",Best Hip-Hop Playlist,NaN,"(We got London on the track!). These days, I d..."


In [130]:
df.to_csv('datasets/checkpoint_lyrics.csv', index = False)

In [61]:
#motivational = ['6Oer1XR6bNH6rMnbPxKzOV']

In [138]:
#scrape_mult(motivational)

In [139]:
#df.shape

In [135]:
#snow = ['4sEKPcBYLcW3woDviiaigj']

In [136]:
#scrape_mult(snow)

In [137]:
#df[df['playlist_id'] == '4sEKPcBYLcW3woDviiaigj']

I was able to increase the number of lyrics scaped by changing redirect max limits. For now, I am moving on to the recommender system and will tackle sentiment analysis later.